In [1]:
import os
import openai
import yaml

import re


import json
import re

# Load API key from a YAML configuration file
with open("config.yaml") as f:
 config_yaml = yaml.load(f, Loader=yaml.FullLoader)

# Initialize the OpenAI client with the API key
client = openai.OpenAI(api_key=config_yaml['token'])

# Set the model name
MODEL = "gpt-4o-mini"

def llm(prompt, stop=["\n"]):
    # Prepare the dialog for the API request
    dialogs = [{"role": "user", "content": prompt}]

    completion = client.chat.completions.create(
        model=MODEL,
        messages=dialogs
    )
    return completion.choices[0].message.content

In [4]:
def generate_sequence_explanation_prompt(original_action):
    
    """
    Generate a prompt for the LLM to think about fine motion control for a given action.
    :param original_action: Perform a signature James Bond pose with a dramatic turn and gunpoint.
    :return: 
    :example:
    The man pivots on his heel, shifting his weight to turn his body sharply while extending his opposite arm outward, creating a dramatic stance.
    The man's wrist flexes as he raises his hand to point the imaginary gun forward, aligning his arm with his shoulder for precision and balance.
    <DONE>
    """
    prompt = f"""
    The action 'original_action: {original_action}' may require detailed control over specific body parts. 
    Please evaluate the action and think carefully about how the movement breaks down into smaller steps. 
    You should independently decide the steps involved in completing this action.

    After thinking, provide a structured list of the steps involved in performing this action.
    
    <original_action>
    {original_action}
    </original_action>
    
    
    <REQUIREMENT>
    - Focus on describing the dynamic movement.
    - Highlight the necessary coordination between body parts.
    -Emphasize the importance of actions: Clearly require that each step must include key movement details, avoiding redundancy. 
    -Streamline the steps: Remind that the generated steps should be merged as much as possible, ensuring each step contains actual dynamic movements rather than empty descriptions.
    
        <FORMAT>
    
        The number of steps should be 1 or 2 or 3 or 4, depending on the TEMPORAL complexity of the action.Do not use too many steps if the action is simple. 2~3 steps are usually enough.  
         
         eg. 'run in a circle with one hand swinging',even if the action is complex by Spatial Composition, it is simple by Temporal. Apparently there is only one step of action and don't need to provide multiple steps. In this case, you can just provide one step. 
          
        For each step, use the words 'The man...'or 'The man's ...(body part)' to describe the action.
        Ensure the explanation is like:
        step1: The ...
        step2: The ...
        ...
        
        </FORMAT>
    </REQUIREMENT>
    

    
    <EXCLUSION>
    - Do not include any description of facial expressions or emotions.
    - Focus solely on the action and movement itself.
    </EXCLUSION>
    
    <END_SIGNAL>
    - When your explanation is finished, signal completion by saying '<SEQUENCEEND>'.
    </END_SIGNAL>

    Now think:
    """
    return prompt

def generate_sequence_explanation_prompt_json(original_action,sequence_explanation_prompt):
    prompt = f"""
<TASK>
Based on your breakdown of the action and the most important original_action, evaluate fine motion control for the following body parts:
 
    <original_action>
    {original_action}
    </original_action>
  
    <breakdown of the action>
    {sequence_explanation_prompt}
    </breakdown of the action>  
    
       
    <BODY_PARTS>
    - spine
    - Left Arm
    - Right Arm
    - Left Leg
    - Right Leg
    </BODY_PARTS>
    

</TASK>



<EXAMPLES>
EXAMPLE1:
{{"body part": "left arm", "description": "The man's left arm remains stationary at his side."}}
{{"body part": "right arm", "description": "The man's right arm moves in a waving motion."}}
{{"body part": "left leg", "description": "The man's left leg is stationary."}}
{{"body part": "right leg", "description": "The man's right leg is lifted slightly off the ground."}}
{{"body part": "spine", "description": "The man's spine moves in a wave-like motion."}}

EXAMPLE2:
{{"body part": "left arm", "description": "The man's left arm is bent at the elbow and held close to his body."}}
{{"body part": "right arm", "description": "The man's right arm moves in a circular motion."}}
{{"body part": "left leg", "description": "The man's left leg is bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee."}}
{{"body part": "spine", "description": "The man's spine moves in a rhythmic motion."}}

EXAMPLE3:
{{"body part": "left arm", "description": "The man's left arm is raised in a bent position."}}
{{"body part": "right arm", "description": "The man's right arm is raised and bent at the elbow."}}
{{"body part": "left leg", "description": "The man's left leg is lifted and bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee and lifted slightly off the ground."}}
{{"body part": "spine", "description": "The spine is arched slightly forward."}}
</EXAMPLES>

<FORMAT>
Ensure the explanation is in the following JSON-like format for each step and body part:



{{
    "step1": [
        {{
            "body part": "left arm",
            "description": "The man's left arm [specific movement description]."
        }},
        {{
            "body part": "right arm",
            "description": "The man's right arm [specific movement description]."
        }},
        {{
            "body part": "left leg",
            "description": "The man's left leg [specific movement description]."
        }},
        {{
            "body part": "right leg",
            "description": "The man's right leg [specific movement description]."
        }},
        {{
            "body part": "spine",
            "description": "The man's spine [specific movement description]."
        }}
    ],
    "step2": [
        {{
            "body part": "left arm",
            "description": "The man's left arm [specific movement description]."
        }},
        {{
            "body part": "right arm",
            "description": "The man's right arm [specific movement description]."
        }},
        {{
            "body part": "left leg",
            "description": "The man's left leg [specific movement description]."
        }},
        {{
            "body part": "right leg",
            "description": "The man's right leg [specific movement description]."
        }},
        {{
            "body part": "spine",
            "description": "The man's spine [specific movement description]."
        }}
    ],
    ...(continue for each step)
}}

Focus on the movement and positioning of each body part, similar to the provided examples. Be concise and avoid vague terms. Use clear and specific descriptions.
</FORMAT>

<REQUIREMENT>
Focus only on these body parts. DO NOT include any details about facial expressions, eyes, or emotions.
Be concise and AVOID providing any reasoning or explanation—focus only on the action of each body part.
</REQUIREMENT>

<END_SIGNAL>
When you finish the explanation for all steps, say '<SEQUENCEEND>'.
</END_SIGNAL>
    """
    


def sequence_analyze(action):
    sequence_explanation_prompt = generate_sequence_explanation_prompt(action)
    sequence_explanation = llm(sequence_explanation_prompt, stop=["<SEQUENCEEND>"]).split("<SEQUENCEEND>")[0].strip()
    print(sequence_explanation)

    # Use the updated format to generate JSON-like output for each body part
    sequence_explanation_prompt2 = generate_sequence_explanation_prompt_json(action, sequence_explanation)
    # print(sequence_explanation_prompt2)
    sequence_explanation2 = llm(sequence_explanation_prompt2, stop=["<SEQUENCEEND>"]).split("<SEQUENCEEND>")[0].strip()

    # Formatting the output into the desired JSON-like format for each body part
    output = (
            "Action: " + "\n" + action + "\n" +
            "Sequence Explanation: " + "\n" + sequence_explanation + "\n" +
            "Fine Motion Control Steps: " + "\n" +
            sequence_explanation2 + "\n" +
            "\n"
    )

    # Write to file
    with open("sequence_explanation.txt", "a") as file:
        file.write(output)

    # Print to console
    print(output)

    # Return results as well
    return sequence_explanation, sequence_explanation2

# txt = sequence_analyze("Perform a signature James Bond pose with a dramatic turn and gunpoint.")
# print("done")

actions_to_test = [
    "Perform a signature James Bond pose with a dramatic turn and gunpoint.",
    "Fight with the fluid precision and power of Bruce Lee.",
    "Perform a graceful spinning kick in the style of Jet Li.",
    "Execute a stealthy crouch and roll like a ninja.",
    "Leap over an obstacle with the agility of a parkour expert.",
    "Perform a slow-motion dive while firing two handguns like in an action movie.",
    "Execute a precise sword slash followed by a defensive stance like a samurai.",
    "Jump from a ledge and roll upon landing to minimize impact, as seen in martial arts films.",
    "Perform a quick disarm move to take an opponent’s weapon away.",
    "Perform a quick, tactical reload of a firearm while maintaining a defensive stance."
]
actions_to_test = [
    "The person performs a rowing motion with their legs spread wide.",
    "A woman hops forward while holding a T-pose.",
    "The man executes a jump spin mid-air.",
    "A person crawls on the ground in a baby-like motion.",
    "A dancer spins gracefully in a ballet twirl.",
    "The computer science student attempts one-armed push-ups.",
    "The soccer player covers their ears during a goal celebration.",
    "A dad crawls on the floor to retrieve his child’s toy.",
    "The police officer sprints to chase someone on foot.",
    "A bodybuilder performs a perfect pistol squat."
]



In [5]:
 
for action in actions_to_test:
    sequence_explanation, sequence_explanation2 = sequence_analyze(action)
   

step1: The man swiftly pivots on his heel, executing a dramatic turn while simultaneously raising his right arm to aim a gun outward.  
step2: The man's left leg steps forward, grounding his stance for balance, as his body angles slightly to the side for a more dynamic pose.  
step3: The man's fingers tighten around the gun's grip, aligning his aim toward the target while his torso remains steady, emphasizing posture and readiness.
Action: 
Perform a signature James Bond pose with a dramatic turn and gunpoint.
Sequence Explanation: 
step1: The man swiftly pivots on his heel, executing a dramatic turn while simultaneously raising his right arm to aim a gun outward.  
step2: The man's left leg steps forward, grounding his stance for balance, as his body angles slightly to the side for a more dynamic pose.  
step3: The man's fingers tighten around the gun's grip, aligning his aim toward the target while his torso remains steady, emphasizing posture and readiness.
Fine Motion Control Steps

In [9]:
def generate_fine_motion_control_prompt(original_action, sequence_explanation):
    prompt = f"""
<TASK>
Generate fine motion control descriptions for each body part based on the action 'original_action' and the sequence_explanation.

<original_action>
{original_action}
</original_action>

<sequence_explanation>
{sequence_explanation}
</sequence_explanation>


</TASK>

<REQUIREMENT>
Focus on describing the specific actions for each body part without providing explanations or reasoning. Avoid vague terms such as "at an appropriate angle" or "adjust for balance." Instead, use precise descriptions like "Raise the right arm to shoulder height." 
Do not include any details about facial expressions, eyes, or emotions.
I don't like overly abstract language, such as "as if aiming a firearm." Understanding this action requires a high level of comprehension that general models may not possess, as this description does not clearly specify what kind of action it is. Please use more specific action descriptions.


</REQUIREMENT>

<EXAMPLES>
EXAMPLE1:
{{"body part": "left arm", "description": "The man's left arm remains stationary at his side."}}
{{"body part": "right arm", "description": "The man's right arm moves in a waving motion."}}
{{"body part": "left leg", "description": "The man's left leg is stationary."}}
{{"body part": "right leg", "description": "The man's right leg is lifted slightly off the ground."}}
{{"body part": "spine", "description": "The man's spine moves in a wave-like motion."}}

EXAMPLE2:
{{"body part": "left arm", "description": "The man's left arm is bent at the elbow and held close to his body."}}
{{"body part": "right arm", "description": "The man's right arm moves in a circular motion."}}
{{"body part": "left leg", "description": "The man's left leg is bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee."}}
{{"body part": "spine", "description": "The man's spine moves in a rhythmic motion."}}

EXAMPLE3:
{{"body part": "left arm", "description": "The man's left arm is raised in a bent position."}}
{{"body part": "right arm", "description": "The man's right arm is raised and bent at the elbow."}}
{{"body part": "left leg", "description": "The man's left leg is lifted and bent at the knee."}}
{{"body part": "right leg", "description": "The man's right leg is bent at the knee and lifted slightly off the ground."}}
{{"body part": "spine", "description": "The spine is arched slightly forward."}}
</EXAMPLES>

<INPUT>
Using the action original_action and the sequence_explanation, generate fine motion control descriptions for the following body parts:

- spine
- left arm
- right arm
- left leg
- right leg

<original_action>
{original_action}
</original_action>

<sequence_explanation>
{sequence_explanation}
</sequence_explanation>

<FORMAT>
For each body part, provide a concise and specific action description in the following JSON format:

{{
    "body part": "left arm",
    "description": "The man's left arm [specific movement description]."
}},
{{
    "body part": "right arm",
    "description": "The man's right arm [specific movement description]."
}},
{{
    "body part": "left leg",
    "description": "The man's left leg [specific movement description]."
}},
{{
    "body part": "right leg",
    "description": "The man's right leg [specific movement description]."
}},
{{
    "body part": "spine",
    "description": "The man's spine [specific movement description]."
}}
</FORMAT>
</INPUT>


<IMPORTANT>
I PERFER the description to be more STRAIGHTFORWARD, USING SIMPLE SENTENCES STRUCTURE, and AVOIDING COMPLEX SENTENCES. For example,  "The man's spine leans back while maintaining tension in the core muscles." should be  "The man's spine leans back " The adding suggestion about state("while maintaining tension in the core muscles.") is too complex and not necessary.
I ONLY NEED THE ACTION DESCRIPTION, NOT THE STATE OR THE SITUATION DESCRIPTION.
For example, "The man's spine leans back slightly during the rowing motion." should be "The man's spine leans back slightly"  The adding suggestion about state("during the rowing motion.") is useless and harmful.
For example, "The woman's left leg is bent slightly at the knee as she prepares to jump." should be "The woman's left leg is bent slightly at the knee "The adding suggestion about state(" as she prepares to jump.") is useless and harmful.
AVOID USING WORDS LIKE "when","as","while","before","after" ,because they are usually used to describe the state or the situation, not the action.

USE SIMPLE SENTENCES STRUCTURE！
USE SIMPLE SENTENCES STRUCTURE！
USE SIMPLE SENTENCES STRUCTURE！
USE SIMPLE SENTENCES STRUCTURE！
</IMPORTANT>

<END_SIGNAL>
When you finish, say '<CONTROLEND>'.
</END_SIGNAL>
"""
    return prompt


def analyze_fine_moton_control_txt(action):
    # Step 1: Get sequence explanation
    sequence_explanation_prompt = generate_sequence_explanation_prompt(action)
    sequence_explanation = llm(sequence_explanation_prompt, stop=["<SEQUENCEEND>"]).split("<SEQUENCEEND>")[0].strip()
    print(sequence_explanation)
    
    
    # Step 2: Evaluate fine motion control
    fine_moton_control_prompt = generate_fine_motion_control_prompt(action, sequence_explanation)
    control_evaluation = llm(fine_moton_control_prompt, stop=["<CONTROLEND>"]).split("<CONTROLEND>")[0].strip()

    # Parse the JSON objects from the control evaluation
    json_objects = re.findall(r'\{[^}]+\}', control_evaluation)
    control_results = [json.loads(obj) for obj in json_objects]

    # Output to file as well as print
    # output = {
    #     "Action": action,
    #     "Sequence Explanation": sequence_explanation,
    #     "Fine Motion Control Evaluation": control_results
    # }
    output = control_results

    # Write to file
    with open("analyze_fine_moton_control.json", "a") as file:
        json.dump(output, file, ensure_ascii=False, indent=2)
        file.write("\n")
        
        
    output2 = {
            "Action": action,
            "Sequence Explanation": sequence_explanation,
            "Fine Motion Control Evaluation": control_results
        }
    with open("fine_motion_control.txt", "a") as file:
        file.write(json.dumps(output2, ensure_ascii=False, indent=2))
        file.write("\n")

    # Print to console
    print(json.dumps(output, ensure_ascii=False, indent=2))

    # Return results as well
    return sequence_explanation, control_results


# actions_to_test = [
#     "Perform a signature James Bond pose with a dramatic turn and gunpoint.",
#     "Fight with the fluid precision and power of Bruce Lee.",
#     "Perform a graceful spinning kick in the style of Jet Li.",
#     # "Execute a stealthy crouch and roll like a ninja.",
#     # "Leap over an obstacle with the agility of a parkour expert.",
#     # "Perform a slow-motion dive while firing two handguns like in an action movie.",
#     # "Execute a precise sword slash followed by a defensive stance like a samurai.",
#     # "Jump from a ledge and roll upon landing to minimize impact, as seen in martial arts films.",
#     # "Perform a quick disarm move to take an opponent’s weapon away.",
#     # "Perform a quick, tactical reload of a firearm while maintaining a defensive stance."
# ]
actions_to_test = [
    "The person performs a rowing motion with their legs spread wide.",
    "A woman hops forward while holding a T-pose.",
    "The man executes a jump spin mid-air.",
    "A person crawls on the ground in a baby-like motion.",
    "A dancer spins gracefully in a ballet twirl.",
    "The computer science student attempts one-armed push-ups.",
    "The soccer player covers their ears during a goal celebration.",
    "A dad crawls on the floor to retrieve his child’s toy.",
    "The police officer sprints to chase someone on foot.",
    "A bodybuilder performs a perfect pistol squat."
]


# Test each action and save the results in the fine_control.txt file
for action in actions_to_test:
    sequence_explanation, control_results = analyze_fine_moton_control_txt(action)
    
    # Append the result to the fine_control.txt file
    with open("fine_motion_control_log.txt", "a") as file:
        # Append both the sequence explanation and control results
        file.write("========================")
        file.write("Action: " + action + "\n")  # Write the action description
        file.write("Sequence Explanation:\n" + sequence_explanation + "\n")  # Write the sequence explanation
        file.write("Control Results:\n" + control_results.__str__() + "\n\n")  # Write the fine motion control results

 

Step 1: The person begins the rowing motion by bending their knees and driving through their heels, while simultaneously leaning back slightly at the hips to engage their core.

Step 2: The person then extends their legs outward, pushing with their feet as they pull their arms back towards their chest, maintaining a wide leg position while coordinating the movement of their arms and torso.
[
  {
    "body part": "left arm",
    "description": "The man's left arm pulls back towards his chest."
  },
  {
    "body part": "right arm",
    "description": "The man's right arm pulls back towards his chest."
  },
  {
    "body part": "left leg",
    "description": "The man's left leg is bent at the knee."
  },
  {
    "body part": "right leg",
    "description": "The man's right leg is bent at the knee."
  },
  {
    "body part": "spine",
    "description": "The man's spine leans back slightly."
  }
]
step1: The woman stands upright with her feet shoulder-width apart and arms extended horizont

In [7]:
def check_trajectory_control_with_llm(action):
    # First call: Let the LLM think using chain of thought reasoning
    first_prompt = f"""
    Examine the action: "{action}".

    Trajectory control is only necessary if the action includes explicit instructions for movement along a specific two-dimensional path. This typically includes instructions like "turn", "move in a circle", "zigzag", or "change direction", which clearly involve a shift in direction or orientation.

    If an action involves simple movement in a straight line (such as "run forward" or "jump in place") or basic rotations (like "spin in place"), trajectory control is not required because there is no complex or changing path to follow.

    Consider whether the action explicitly requires a change in direction or follows a curved or angular path, then conclude if trajectory control is necessary. If the path is simple or linear, trajectory control is not needed.

    Explain your reasoning step by step and conclude with whether trajectory control is required.
    """

    first_response = llm(first_prompt)

    # Second call: Ask LLM to give a simple boolean response based on its reasoning
    second_prompt = f"""
Based on your previous analysis: "{first_response}", answer with only 'True' if trajectory control is required, and 'False' if it is not.
"""
    second_response = llm(second_prompt).strip()

    return second_response


# Convert the second response to a boolean value
def response_to_bool(response):
    if response.lower() == "true":
        return True
    elif response.lower() == "false":
        return False
    else:
        raise ValueError("Unexpected response from LLM: " + response)

#
# # List of action cases to test
# actions = [
#     "Run forward, then jump in place",                       # Example 1
#     "Walk forward and make a left turn",                     # Example 2
#     "Stand still and wave your hand",                        # Example 3
#     "Run up, then kneel and slide backward",                 # Example 4
#     "Walk around in a circle",                               # Example 5
#     "Spin in place on one foot",                             # Example 6
#     "Crawl under the table",                                 # Example 7
#     "Run forward and stop at the door",                      # Example 8
#     "Jump forward and then turn around",                     # Example 9
#     "Dance freely with no set path",                         # Example 10
#     "Climb up the ladder",                                   # Example 11
#     "Roll on the ground twice",                              # Example 12
#     "Run forward, kneel, and slide in a straight line",      # Example 13
#     "Hop in a zigzag pattern",                               # Example 14
#     "Jump sideways then spin 180 degrees",                   # Example 15
#     "Walk straight then stop and jump on one leg",           # Example 16
#     "Swim forward in a straight line",                       # Example 17
#     "Skate in a figure-eight pattern",                       # Example 18
#     "Step backward three times then kneel",                  # Example 19
#     "Jump forward and spin in a circle mid-air"              # Example 20
# ]
#
# # Test trajectory control for each action using LLM
# for action in actions:
#     print(f"Action: {action}")
#     reasoning = check_trajectory_control_with_llm(action)
#     result_bool = response_to_bool(reasoning)
#     print(f"Needs Trajectory Control: {result_bool}")
#     print("=======================================")


In [18]:
def generate_comparison_feedback_prompt(description1, description2):
    prompt = f"""
<TASK>
You are tasked with analyzing two motion descriptions, `description1` (generated by a model) and `description2` (a gold-standard or reference description). Your goal is to carefully compare the two descriptions for each of the following body parts: spine, left arm, right arm, left leg, and right leg. For each body part, identify any discrepancies between the two descriptions and provide a single piece of feedback to help improve `description1` so it more closely matches `description2`. It is better to use standard command words from the Standard_Manual.

<BODY_PARTS>
- spine
- Left Arm
- Right Arm
- Left Leg
- Right Leg
</BODY_PARTS>

<description1>
{description1}
</description1>

<description2>
{description2}
</description2>

<Standard_Manual>
**Standard Manual for Human Motion Description**

This manual aims to standardize the language used by models when describing human poses and motions, making it more precise and specific while avoiding vague and abstract expressions. By employing a unified set of terms, phrases, and descriptive standards, it ensures that the content generated by the model is clear and consistent, making it easier for readers to understand and apply.

---

### **1. Introduction**

In the fields of artificial intelligence and computer vision, accurately describing human motion is crucial for the training and application of models. This manual provides a comprehensive and detailed set of guidelines to assist models in generating precise human motion descriptions, avoiding ambiguity and vagueness.

### **2. Basic Principles**

- **Precision**: Descriptions should be as specific as possible, providing clear angles, directions, positions, and motion details.
- **Consistency**: Use a uniform set of terms and phrases to ensure the descriptions are coherent and comprehensible.
- **Objectivity**: Avoid subjective evaluations or emotional tones, focusing on the statement of objective facts.
- **Conciseness**: Expressions should be brief and to the point, highlighting key information while avoiding redundancy.

### **3. Description Standards**

#### **3.1 Angle and Direction Descriptions**

**Angle Categories:**

- **Vertical**: Angle ≤ 10°
  - **Example**: *“The arm is raised vertically.”*
- **Near Vertical**: 10° < Angle ≤ 30°
  - **Example**: *“The thigh is nearly vertical to the ground.”*
- **Diagonal**: 30° < Angle ≤ 60°
  - **Example**: *“The arm extends at a 45-degree angle to the front side.”*
- **Near Horizontal**: 60° < Angle ≤ 80°
  - **Example**: *“The lower leg is nearly horizontal.”*
- **Horizontal**: Angle > 80°
  - **Example**: *“The arms are extended horizontally to the sides.”*

**Direction Descriptions:**

- **Front, Back, Left, Right, Up, Down**
  - **Example**: *“The head turns to the left.”*, *“The leg steps forward.”*
- **Inner, Outer**
  - **Example**: *“The palm rotates inward.”*, *“The toes point outward.”*
- **Clockwise, Counterclockwise**
  - **Example**: *“The arm rotates in a clockwise direction.”*

#### **3.2 Motion Type Descriptions**

- **Static Motion**: Maintaining a pose without movement.
  - **Example**: *“He stands still with his hands by his sides.”*
- **Continuous Motion**: Motion ongoing without interruption.
  - **Example**: *“She continues to run with a steady pace.”*
- **Repetitive Motion**: Actions that are repeated in a regular pattern.
  - **Example**: *“He jumps up and down repeatedly.”*
- **Single Motion**: An action that is completed once.
  - **Example**: *“She raises her arm and waves.”*

#### **3.3 Motion Amplitude and Degree Descriptions**

- **Complete**: The action reaches its maximum range.
  - **Example**: *“The knee is bent completely to bring the calf against the thigh.”*
- **Large Amplitude**: The action is close to its maximum range.
  - **Example**: *“He swings his arm widely.”*
- **Medium Amplitude**: The action reaches a general range.
  - **Example**: *“She raises her leg at a moderate height.”*
- **Small Amplitude**: The action has a small range.
  - **Example**: *“He slightly shakes his head.”*
- **Slightly**: The action amplitude is minimal, with little change.
  - **Example**: *“She slightly lifts her chin.”*

#### **3.4 Time and Frequency Descriptions**

- **One-time**: The action occurs only once.
  - **Example**: *“He presses the button.”*
- **Continuous**: The action is performed uninterrupted over time.
  - **Example**: *“She is continuously stretching her body.”*
- **Intermittent**: The action occurs at intervals.
  - **Example**: *“He intermittently types on the keyboard.”*
- **Frequent**: The action repeats multiple times in a short period.
  - **Example**: *“She blinks frequently.”*
- **Slow**: The action is performed at a slow pace.
  - **Example**: *“He slowly raises his arm.”*
- **Fast**: The action is performed at a fast pace.
  - **Example**: *“She quickly turns around.”*

#### **3.5 Position and Spatial Relationship Descriptions**

- **Contact**: Between body parts or with an object.
  - **Example**: *“The palm is pressed against the wall.”*
- **Distance**: Describing the space between two body parts or objects.
  - **Close**: Distance ≤ 0.2 meters
    - **Example**: *“The feet are close together.”*
  - **Apart**: 0.2 meters < Distance ≤ 0.5 meters
    - **Example**: *“The hands are shoulder-width apart.”*
  - **Far Apart**: Distance > 0.5 meters
    - **Example**: *“The arms are extended far from the body.”*
- **Height**: Relative to the ground or other reference points.
  - **Below, Equal to, Above**
    - **Example**: *“The shoulder height is equal to the table height.”*

### **4. Terms and Definitions**

#### **4.1 Body Part Names**

```
<BODY_PARTS>
- spine
- Left Arm
- Right Arm
- Left Leg
- Right Leg
</BODY_PARTS>
```

#### **4.2 Action Verbs**

- **Basic Actions**: Raise, lower, bend, extend, rotate, kick, grasp, push, pull, jump, squat, stand, sit, lie
- **Compound Actions**: Step over, squat down, flip forward, flip backward, kick leg, stretch, twist

#### **4.3 Directional Terms**

- **Horizontal Directions**: Front, back, left, right
- **Vertical Directions**: Up, down
- **Spatial Relationships**: Inner, outer, front, back, side, opposite

#### **4.4 Angle and Amplitude Terms**

- **Angle Unit**: Degrees (°)
- **Angle Descriptions**: Vertical, horizontal, diagonal, right angle (90°)
- **Amplitude Descriptions**: Complete, large amplitude, medium amplitude, small amplitude, slight



### **7. Notes**

#### **7.1 Avoid Vague and Subjective Descriptions**

- **Non-standard**: *“He moves in a strange way.”*
- **Standard**: *“He crosses his left foot over his right foot and leans to the right.”*

#### **7.2 Ensure Consistency and Precision**

- Always use the terms and standards outlined in this manual, avoiding different terms for the same entity.



### **8. Appendix**

#### **8.1 Common Angle and Distance Reference Table**

| Angle Description | Range     |
| ----------------- | --------- |
| Vertical          | 0° - 10°  |
| Near Vertical     | 10° - 30° |
| Diagonal          | 30° - 60° |
| Near Horizontal   | 60° - 80° |
| Horizontal        | 80° - 90° |

| Distance Description | Range                   |
| -------------------- | ----------------------- |
| Close                | ≤ 0.2 meters            |
| Apart                | 0.2 meters - 0.5 meters |
| Far Apart            | > 0.5 meters            |

 

</Standard_Manual>

<INSTRUCTIONS>

Editing command: to help improve `description1` so it more closely matches `description2`.
1. **Comparison**:
    - Compare the descriptions for each body part mentioned above. Focus on specific aspects of the motion, such as:
        - **Movement**: How the body part is moving (e.g., raised, lowered, rotated).
        - **Direction**: In which direction the movement occurs (e.g., upward, downward, forward, backward).
        - **Intensity**: Is the movement fast or slow? Sudden or smooth?
        - **Positioning**: The final position of the body part (e.g., shoulder height, bent at the knee).
    - If a body part is mentioned in `description2` but not in `description1`, note that as a discrepancy.

2. **Feedback**:
    - For each body part, if a discrepancy is found, generate **one** piece of feedback that includes:
        - **Body part**: Identify which body part the feedback is for (e.g., left arm, right leg, spine).
        - **Issue**: Briefly describe the difference between `description1` and `description2` for this body part.
        - **Suggested Change**: Provide a clear, actionable suggestion to adjust `description1` to match `description2`. Be specific about how the movement should be changed.
    - Ensure that only one piece of feedback is given for each body part, even if there are multiple issues. Prioritize the most significant discrepancy.

3. **Output**:
    - Format the feedback as follows:
    {{
        "body part": "spine",
        "issue": "[description of discrepancy in spine movement]",
        "editing command": "The man..../ The man's spine ... [specific suggestion to correct the spine movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "left arm",
        "issue": "[description of discrepancy in left arm movement]",
        "suggestion": "The man..../ The man's left arm ... [specific suggestion to correct the left arm movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "right arm",
        "issue": "[description of discrepancy in right arm movement]",
        "suggestion": "The man..../ The man's right arm ... [specific suggestion to correct the right arm movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "left leg",
        "issue": "[description of discrepancy in left leg movement]",
        "suggestion": "The man..../ The man's left leg ... [specific suggestion to correct the left leg movement,should use Standard_Manual style of words]"
    }},
    {{
        "body part": "right leg",
        "issue": "[description of discrepancy in right leg movement]",
        "suggestion": "The man..../ The man's right leg ... [specific suggestion to correct the right leg movement,should use Standard_Manual style of words]"
    }}

4. **Exclusions**:
    - Do not analyze facial expressions, emotions, or any abstract descriptions.
    - Focus only on the physical actions of the body parts mentioned in `<BODY_PARTS>`.

<END_SIGNAL>
When you have finished generating the feedback, end the output with '<FEEDBACKEND>'.
</END_SIGNAL>
</INSTRUCTIONS>

Now, compare the two descriptions and generate feedback.
    """
    return prompt



In [19]:
import json
import re

def test_generate_comparison_feedback(description1, description2):
    # 生成对比反馈的 prompt
    comparison_prompt = generate_comparison_feedback_prompt(description1, description2)
    
    comparison_feedback = llm(comparison_prompt, stop=["<END_FEEDBACK>"]).split("<END_FEEDBACK>")[0].strip()
    
    json_objects = re.findall(r'\{[^}]+\}', comparison_feedback)
    feedback_results = [json.loads(obj) for obj in json_objects]
    
    # 保存到文件中
    output_filename = "comparison_feedback_results.json"
    with open(output_filename, "w") as file:
        json.dump(feedback_results, file, ensure_ascii=False, indent=2)
    
    # 打印输出
    print(f"Comparison Feedback Results saved to {output_filename}")
    print(json.dumps(feedback_results, ensure_ascii=False, indent=2))
    
    # 返回生成的反馈结果
    return feedback_results



In [20]:
# 金标准和带偏差的动作描述


# 第二个金标准和带偏差的动作描述
description12= """
[
    {"body part": "left arm", "description": "The man's left arm is straightened out and slightly away from his body."},
    {"body part": "right arm", "description": "The man's right arm moves in a sharp back-and-forth motion."},
    {"body part": "left leg", "description": "The man's left leg is straight and does not bend."},
    {"body part": "right leg", "description": "The man's right leg is slightly bent but not fully."},
    {"body part": "spine", "description": "The man's spine is slightly curved forward."}
]
"""

description22 = """
[
    {"body part": "left arm", "description": "The man's left arm is bent at the elbow and held close to his body."},
    {"body part": "right arm", "description": "The man's right arm moves in a circular motion."},
    {"body part": "left leg", "description": "The man's left leg is bent at the knee."},
    {"body part": "right leg", "description": "The man's right leg is bent at the knee."},
    {"body part": "spine", "description": "The man's spine moves in a rhythmic motion."}
]
"""

# 第三个金标准和带偏差的动作描述
description13 = """
[
    {"body part": "left arm", "description": "The man's left arm is fully extended above his head."},
    {"body part": "right arm", "description": "The man's right arm remains straight and is raised to shoulder height."},
    {"body part": "left leg", "description": "The man's left leg remains straight and does not lift."},
    {"body part": "right leg", "description": "The man's right leg is straight and lifted off the ground."},
    {"body part": "spine", "description": "The man's spine remains straight without arching."}
]
"""

description23 = """
[
    {"body part": "left arm", "description": "The man's left arm is raised in a bent position."},
    {"body part": "right arm", "description": "The man's right arm is raised and bent at the elbow."},
    {"body part": "left leg", "description": "The man's left leg is lifted and bent at the knee."},
    {"body part": "right leg", "description": "The man's right leg is bent at the knee and lifted slightly off the ground."},
    {"body part": "spine", "description": "The spine is arched slightly forward."}
]
"""

# 测试生成反馈
test_generate_comparison_feedback(description12, description22)




Comparison Feedback Results saved to comparison_feedback_results.json
[
  {
    "body part": "spine",
    "issue": "The man's spine is described as slightly curved forward in description1, whereas description2 states that the spine moves in a rhythmic motion.",
    "editing command": "The man's spine moves in a rhythmic motion."
  },
  {
    "body part": "left arm",
    "issue": "The man's left arm is described as straightened out and slightly away from his body in description1, but it is actually bent at the elbow and held close to his body in description2.",
    "editing command": "The man's left arm is bent at the elbow and held close to his body."
  },
  {
    "body part": "right arm",
    "issue": "The man's right arm is described as moving in a sharp back-and-forth motion in description1, while description2 specifies that it moves in a circular motion.",
    "editing command": "The man's right arm moves in a circular motion."
  },
  {
    "body part": "left leg",
    "issue": "Th

[{'body part': 'spine',
  'issue': "The man's spine is described as slightly curved forward in description1, whereas description2 states that the spine moves in a rhythmic motion.",
  'editing command': "The man's spine moves in a rhythmic motion."},
 {'body part': 'left arm',
  'issue': "The man's left arm is described as straightened out and slightly away from his body in description1, but it is actually bent at the elbow and held close to his body in description2.",
  'editing command': "The man's left arm is bent at the elbow and held close to his body."},
 {'body part': 'right arm',
  'issue': "The man's right arm is described as moving in a sharp back-and-forth motion in description1, while description2 specifies that it moves in a circular motion.",
  'editing command': "The man's right arm moves in a circular motion."},
 {'body part': 'left leg',
  'issue': "The man's left leg is described as straight and does not bend in description1, but in description2, it states that the le